In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/datasets/dataset_B_05_2020.csv')
data=data.drop(['url'],axis=1)
print(data.head())

# x=np.array(data.drop(['status'],axis=1))
# y=data['status']
# y=[1 if y[i]=='legitimate' else -1 for i in range(len(y))]
# y=np.array(y)
# print(len(x[y==1]),len(x[y==-1]))
# train_x=x[y==1][1000:]
# train_x=np.append(train_x,x[y==-1][1000:],axis=0)
# train_y=x[y==0][1000:]


   length_url  length_hostname  ip  nb_dots  nb_hyphens  nb_at  nb_qm  nb_and  \
0          37               19   0        3           0      0      0       0   
1          77               23   1        1           0      0      0       0   
2         126               50   1        4           1      0      1       2   
3          18               11   0        2           0      0      0       0   
4          55               15   0        2           2      0      0       0   

   nb_or  nb_eq  ...  domain_in_title  domain_with_copyright  \
0      0      0  ...                0                      1   
1      0      0  ...                1                      0   
2      0      3  ...                1                      0   
3      0      0  ...                1                      0   
4      0      0  ...                0                      1   

   whois_registered_domain  domain_registration_length  domain_age  \
0                        0                          45    

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('status', axis=1), data['status'], test_size=0.2, random_state=42, stratify=data['status'])


In [4]:
variances = np.var(X_train-np.mean(X_train), axis=0)

print("Variance for each dimension:", variances)

Variance for each dimension: length_url         3.188433e+03
length_hostname    1.073398e+02
ip                 1.267640e-01
nb_dots            1.924850e+00
nb_hyphens         4.084721e+00
                       ...     
domain_age         9.598391e+06
web_traffic        4.036886e+12
dns_record         1.971757e-02
google_index       2.490690e-01
page_rank          6.405836e+00
Length: 87, dtype: float64


In [6]:
print("Train set shape:", X_train.shape, X_test.shape)
y_train=np.array(y_train)
y_test=np.array(y_test)
y_train=np.array([1 if y_train[i]=='legitimate' else 0 for i in range(len(y_train))])
y_test=np.array([1 if y_test[i]=='legitimate' else 0 for i in range(len(y_test))])
X_train=np.array(X_train)
X_test=np.array(X_test)
print("Test set shape:",y_train.shape, y_test.shape)
print("Train set shape:", X_train.shape, X_test.shape)

x_train_t=X_train.T

Train set shape: (9144, 87) (2286, 87)
Test set shape: (9144,) (2286,)
Train set shape: (9144, 87) (2286, 87)


In [7]:
print(y_test)
print(y_train)

[0 0 0 ... 1 1 1]
[1 0 0 ... 0 0 0]


In [8]:
regions=[]
for i in range(len(x_train_t)):
  regions.append([min(x_train_t[i]),max(x_train_t[i])])

print(regions)
boundaries=[]
def validPointForTrain(obj,regions):
  for i in range(len(regions)):
    # print(obj[i],regions[i][0],regions[i][1])
    if obj[i]<regions[i][0] or obj[i] > regions[i][1]:
      return False
  return True
def validPointForTest(obj,regions):

  for i in regions:
    # print(obj[i],regions[i][0],regions[i][1])
    #(dim,mid,l/r)
    if (i[2]=='l' and obj[i[0]] > i[1]):
      return False
    if (i[2]=='r' and obj[i[0]] < i[1]):
      return False
  return True
intvls=[]
def learn_tree(y,yT,data,nodes,lab,dim,regions,decisions,boundaries):
  # print(nodes)
  min_g=()
  for j in range(dim):
    d=y[j]
    sorted_d=sorted(d)
    me=0
    cnt=0
    x=(regions[j][0]+regions[j][1])/2
    # print(x)
    # print(regions[j][0],regions[j][1],x)

    p=[0,0]
    q=[0,0]
    for k in range(len(lab)):
      if not validPointForTest(yT[k],boundaries):
        continue
        # print(k)
        # return
      if yT[k][j]<x:
        p[lab[k]]+=1
      else:
        q[lab[k]]+=1
    # print("lengths:",len(p),len(q))
    # print(p,x)
    # print(p,q)
    ps=sum(p)
    qs=sum(q)
    # if(ps==0 and qs==0 ):
    #   print(p,q)
    #   continue
    # print(p,q)
    gp=0
    gq=0
    # print(len(p),len(q),ps,qs)
    for k in range(2):
      p[k]/=1 if ps==0 else ps
      # print(ps,qs,k)
      q[k]/=1 if qs==0 else qs
      gp+=p[k]*(1-p[k])
      gq+=q[k]*(1-q[k])
    g=(gp*ps+gq*qs)/(ps+qs)
    pi=0
    pv=p[0]
    qi=0
    qv=q[0]
    for k in range(1,2):
      if p[k]>pv:
        pv=p[k]
        pi=k
      if q[k]>qv:
        qv=q[k]
        qi=k
    if j==0 or g<min_g[0]:
      min_g=(g,j,gp,gq,pi,qi,x)
  b1=[i for i in boundaries]



  b1.append((min_g[1],min_g[-1],'l'))
  # print("min_g",min_g,nodes)
  b2=[i for i in boundaries]
  b2.append((min_g[1],min_g[-1],'r'))

  r1=[[i,j] for i,j in regions]
  r2=[[i,j] for i,j in regions]
  r1[min_g[1]][1]=min_g[-1]
  r2[min_g[1]][0]=min_g[-1]+1
  if(nodes==2):

    decisions.append((b2,min_g[5]))
    decisions.append((b1,min_g[4]))

    return
  # return
  # print(min_g)
  if(min_g[2]<min_g[3]):
    decisions.append((b1,min_g[4]))
    # decisions.append((b2,min_g[5]))

    learn_tree(y,yT,data,nodes-1,lab,dim,r2,decisions,b2)
    # learn_tree(y,yT,data,nodes-1,lab,10,r1,decisions,b1)

  else:
    decisions.append((b2,min_g[5]))
    # decisions.append((b2,min_g[5]))

    learn_tree(y,yT,data,nodes-1,lab,dim,r1,decisions,b1)

decisions=[]
boundaries=[]
learn_tree(x_train_t,X_train,data,10,y_train,len(x_train_t),regions,decisions,boundaries)
print(decisions)
# print([i[1] for i in decisions])
totals=[0,0]
corrects=[0,0]
def find_class(decisions,elem):
  ans=-1
  for i in decisions:
    if validPointForTest(elem,i[0]):
      ans=i[1]
  if(ans==-1):
    print(ans)
  return ans
# decisions.append(([(85, 0.5, 'r')], 0))
# print(test_y.shape,test_labls.shape)



[[12.0, 1641.0], [4.0, 214.0], [0.0, 1.0], [1.0, 24.0], [0.0, 32.0], [0.0, 4.0], [0.0, 3.0], [0.0, 19.0], [0.0, 0.0], [0.0, 19.0], [0.0, 18.0], [0.0, 1.0], [0.0, 54.0], [2.0, 33.0], [0.0, 1.0], [1.0, 7.0], [0.0, 4.0], [0.0, 20.0], [0.0, 6.0], [0.0, 18.0], [0.0, 2.0], [0.0, 6.0], [0.0, 1.0], [0.0, 4.0], [0.0, 1.0], [0.0, 0.7238805970000001], [0.0, 0.8], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 3.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 6.0], [0.0, 1.0], [1.0, 96.0], [0.0, 145.0], [1.0, 31.0], [1.0, 39.0], [0.0, 40.0], [2.0, 829.0], [1.0, 61.0], [0.0, 829.0], [2.0, 106.5], [1.0, 39.0], [0.0, 206.0], [0.0, 10.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 2.0], [0.0, 4659.0], [0.0, 1.0], [0.0, 1.0], [0.0, 0.0], [0.0, 124.0], [0.0, 0.0], [0.0, 2.0], [0.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 100.0], [0.0, 0.0], [0.0, 100.0], [0.0, 100.0], [0.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 100.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.

In [9]:
import random
max_acc=[0,[]]

for w in range(1):

  d5=[[],[],[],[],[],[],[],[]]
  l5=[[],[],[],[],[],[],[],[]]
  # for i in range(len(all_lab)):
  #   for j in range(5):
  #     idx=random.randint(0,len(all_lab)-1)
  #     d5[j].append(all_data[idx]);
  #     l5[j].append(all_lab[idx]);
  ids=[i for  i in range(len(y_train))]
  for i in range(8):
    idxs=random.choices(ids,k=len(y_train))
    d5[i]=[X_train[j] for j in idxs]
    l5[i]=[y_train[j] for j in idxs]
  print(len(d5[0]),len(d5[1]),len(d5[2]),len(d5[3]),len(d5[4]))

  trees=[]
  for j in range(8):
    d=d5[j]
    d=np.array(d)
    dT=d.T
    regions=[]
    for i in range(len(dT)):
      regions.append([min(dT[i]),max(dT[i])])
    # print(regions)
    tree=[]

    learn_tree(dT,d,d,12,l5[j],len(dT),regions,tree,[])

    trees.append(tree)
  # test=[]
  # test_labls=[]

  # corrects=[0,0]
  # totals=[0,0]
  # print(len(y_test))
  # for i in range(len(y_test)):
  #   cl=[0,0,0]
  #   for j in range(5):
  #     cl[find_class(trees[j],X_test[i])]+=1
  #   # print(cl)
  #   freq=0
  #   id=-1
  #   for j in range(3):
  #     if cl[j]>freq:
  #       freq=cl[j]
  #       id=j

  #   if y_test[i]==id:
  #     corrects[y_test[i]]+=1
  #   totals[y_test[i]]+=1

  # print(sum(corrects)/sum(totals))










9144 9144 9144 9144 9144


In [10]:
corrects=[0,0]
totals=[0,0]
print(len(y_test))
for i in range(len(y_test)):
  cl=[0,0,0]
  for j in range(7):
    cl[find_class(trees[j],X_test[i])]+=1
  # print(cl)
  freq=0
  id=-1
  for j in range(3):
    if cl[j]>freq:
      freq=cl[j]
      id=j
  if y_test[i]==id:
    corrects[y_test[i]]+=1
  totals[y_test[i]]+=1

tp=corrects[0]
fp=totals[1]-corrects[1]
fn=totals[0]-corrects[0]
tn=corrects[1]
precision=tp/(tp+fp)
recall=tp/(tp+fn)
print("Precision:",precision*100,"%")
print("Recall:",recall*100,"%")
print("F1-Score:",100*2*precision*recall/(precision+recall),"%")
print("Accuracy:",100*(tp+tn)/len(y_test),"%")
print(sum(corrects)/sum(totals))


2286
Precision: 88.87923544743701 %
Recall: 89.501312335958 %
F1-Score: 89.1891891891892 %
Accuracy: 89.15135608048993 %
0.8915135608048994
